## Children Act timeliess

In [ ]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re
import os  # for file paths
import awswrangler as wr

In [ ]:
# Structured  dataframes
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

##### Assigning key variables

In [ ]:
# define some variables to be used throughout the notebook
db1 = "familyman_derived_dev_v2" #database where Familyman data is stored
db2 = "familyman_dev_v3"
snapshot_date = "2022-08-04" #To update where necessary
db3 = "fcsq" #database where tables created as part of FCSQ processing are stored where required

#### Extract date of issue

In [ ]:
pydb.create_temp_table(
f""" 
SELECT 
  case_number,
  CAST(value AS Date) AS case_DOI
FROM 
  {db2}.case_fields AS f
 
WHERE
  field_model = 'FM2C_DI'
  AND mojap_snapshot_date = DATE'{snapshot_date}'

""",

"DOI")

#### Create start date using date of issue

In [ ]:
#Adding the start date to the main child apps data created in the main child apps process (so data is at child and order level)
#Only including specific order types
pydb.create_temp_table(
f""" 
SELECT 
  t1.case_number,
  t1.receipt_date,
  t1.event,
  t1.field_model,
  t1.order_type,
  t1.order_code,
  t1.order_desc,
  t1.child_role_id,
  t1.order_case_type,
  t2.case_DOI,
  CASE WHEN  t2.case_DOI is null
        THEN t1.receipt_date
       WHEN t1.field_model = 'U22_AT' AND (t2.Case_DOI<t1.Receipt_date)
        THEN t2.Case_DOI
        ELSE t1.Receipt_date END
      AS Start_date
FROM 
  {db3}.ca_apps_child AS t1
      LEFT JOIN __temp__.DOI AS t2 
       ON t1.case_number = t2.case_number
WHERE
  t1.order_code IN (1,4,14,25,27,29,30,31,32)

""",

"child_doi")

#### Matching applications to disposals

In [ ]:
#Matching apps data to the main child disposal data set created in the child disposals processing - matches at child and order level
#Filetering out certain order types alongside interim orders
#Ranks the earlies diposal as the first disposal
pydb.create_temp_table(
f""" 
SELECT 
  a.*,
  YEAR(d.disp_date) AS Year,
  QUARTER(d.disp_date) AS Quarter,
  MONTH(d.disp_date) AS Month,
  d.disp_date,
  date_diff('day',a.start_date, d.disp_date) AS days,
  date_diff('week',a.start_date, d.disp_date) AS weeks,
  ROW_NUMBER() OVER(PARTITION BY a.case_number, a.child_role_id, a.order_code, a.start_date
                       ORDER BY d.disp_date) 
      AS disp_rank   
FROM
  __temp__.child_doi a
  LEFT JOIN __temp__.ca_disps_all_children d
    ON a.case_number = d.case_number
    AND a.order_case_type = d.order_case_type
    AND a.child_role_id = d.child_role_id
WHERE 
  date_diff('day',a.start_date, d.disp_date) >= 0
  AND d.event_model NOT IN ('CPA','C21','C27','C30','C31','C33','C35B','C44A','C44B','C46A','C46B','C47A','C47C','C48A', 
                          'C48B','C48C','C49','D51','D84C','MAGEPO','MAGS37')
  AND d.disp_type_code <> 2

""",

"app_disp_match")

#### Selecting the earliest disposal

In [ ]:
#Selects the first diposal for each order type/child matched
#Multiple children can get matched to the same disposal (if the disposal has multiple child id's)
#More than one application/order type can be matched to one disposal - here we are replicating the SAS process 
pydb.create_temp_table(
f""" 
SELECT 
  *
FROM 
  __temp__.app_disp_match
WHERE
  disp_rank = 1

""",

"first_disp")

#### Creation of main timeliess dataset, deletion of dup apps

In [ ]:
#Here we get rid of any instances of an application of the same order type for the same child, and only the earliest one is selected
pydb.create_temp_table(
f""" 
WITH dup_app_type AS
 (SELECT 
   *,
   ROW_NUMBER() OVER(PARTITION BY case_number, child_role_id, order_code
                       ORDER BY start_date, case_number, child_role_id, order_code) 
     AS dup_app_rank
 FROM 
   __temp__.first_disp
 )

SELECT
  *
FROM
  dup_app_type
WHERE
  dup_app_rank = 1

""",

"ca_timeliness_all")

#### Legal rep

##### Applicants

In [ ]:
#Table of case numbers in which at least one applicant has legal rep
pydb.create_temp_table(
f""" 
SELECT 
  DISTINCT
    case_number,
    legal_rep
FROM 
  {db3}.ca_applicants
WHERE
  legal_rep = 'Y'

""",

"app_legal_rep")

##### Respondents

In [ ]:
#Table of case numbers in which at least one respondent has legal rep
pydb.create_temp_table(
f""" 
SELECT 
  DISTINCT
    case_number,
    legal_rep
FROM 
  {db3}.ca_respondents
WHERE
  legal_rep = 'Y'

""",

"resp_legal_rep")

#### Timeliness with legal rep

In [ ]:
pydb.create_temp_table(
f""" 
SELECT 
  t.*,
  CASE WHEN a.legal_rep = 'Y'
       AND  r.legal_rep = 'Y'
         THEN 'Both'
       WHEN a.legal_rep = 'Y'
       AND  r.legal_rep IS NULL
         THEN 'Applicant Only'
       WHEN a.legal_rep IS NULL
       AND  r.legal_rep = 'Y'
         THEN 'Respondent Only' 
      WHEN a.legal_rep IS NULL
      AND  r.legal_rep IS NULL
         THEN 'Neither' END
    AS leg_rep_cat 
FROM 
  __temp__.ca_timeliness_all t
  LEFT JOIN __temp__.app_legal_rep a
    ON t.case_number = a.case_number
  LEFT JOIN __temp__.resp_legal_rep r
    ON t.case_number = r.case_number

""",

"timeliness_legal_rep")

In [ ]:
pydb.create_temp_table(
f""" 
 SELECT
   year,
   leg_rep_cat,
   count(case_number) AS disposals,
   AVG(days)/7 as av_weeks
 FROM
   __temp__.timeliness_legal_rep
 WHERE 
   order_case_type = 'P'
   and year > 2010
GROUP BY
  year,
  leg_rep_cat
""",

"avg_time")

In [ ]:
pydb.read_sql_query ("select * from __temp__.avg_time order by year,leg_rep_cat")

In [ ]:
data = pydb.read_sql_query ("select * from __temp__.qtr_time")

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df.to_csv ('Exports/t10_checks.csv',header = True)